In [1]:
pip install selenium pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

# Setup Chrome options
options = Options()
options.add_argument("--start-maximized")  # optional
# options.add_argument("--headless")  # uncomment to run headless

# Start the driver
driver = webdriver.Chrome(options=options)

# Go to Google Maps search result of a place
search_query = "Suka+Dessert/@3.2414465,101.6493204,17z/data=!4m8!3m7!1s0x31cc47f4c5890dbd:0x19f2a665e68874fe!8m2!3d3.2414465!4d101.6493204!9m1!1b1!16s%2Fg%2F11h2c1vbtp!5m1!1e1?entry=ttu&g_ep=EgoyMDI1MDQwNi4wIKXMDSoASAFQAw%3D%3D"
driver.get(f"https://www.google.com/maps/place/{search_query}")

time.sleep(5)  # wait for the page to load

# Click on the reviews button
try:
    reviews_button = driver.find_element(By.CSS_SELECTOR, '[aria-label="Reviews for Suka Dessert"]')
    reviews_button.click()
    time.sleep(1)
except Exception as e:
    print("Couldn't click reviews button:", e)
    driver.quit()
    exit()  # Exit the script if the button couldn't be clicked

# Scroll to load more reviews
scrollable_div = driver.find_element(By.CSS_SELECTOR, ".k7jAl.miFGmb.lJ3Kh.PLbyfe")

while True:
    # Get the initial scroll height
    last_scroll_height = driver.execute_script('return arguments[0].scrollHeight;', scrollable_div)

    # Scroll to the bottom of the div
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight;', scrollable_div)

    # Wait for new content to load (adjust as needed)
    time.sleep(2)

    # Get the new scroll height
    new_scroll_height = driver.execute_script('return arguments[0].scrollHeight;', scrollable_div)

    # If the scroll height hasn't changed, we've reached the bottom
    if new_scroll_height == last_scroll_height:
        break

# Extract reviews
review_elems = driver.find_elements(By.XPATH, '//div[@class="jftiEf fontBodyMedium"]')

for i, review in enumerate(review_elems):
    try:
        text = review.find_element(By.CLASS_NAME, 'wiI7pd').text
        print(f"{i+1}. {text}\n")
    except Exception as e:
        print(f"Could not extract text from review {i+1}: {e}")
        continue

# Close the browser
driver.quit()